# Non-stationary Contextual Multi-armed Bandit
## Introduction

- Contextual MAB aims to find the best policy given context features as side information.
- One key assumption: the underlying relationship between context features and rewards are fixed or move very slowly.

### Problem
- In many real world situations, this is not the case. 
- Factors can change suddenly and dramatically (non-stationary)

### Examples
- a user's interest in watching the NBA would suddenly drop in June when the season is over. 
- a user moved to another city or country.

Therefore, in these situations, many alogrithms will be stuck in sub-optimal decisions for a long-time or forever.

### Demonstration
This notebook implemented a popular contextual bandit algorithm: LinUCB
And applied this algorithm in both a stationary and a non-stationary environment.
The result plots can be found at very bottom.
We can see that in this simulation setup, given a stationary environment, our policy converges to optimal in about 20 timesteps. However, given the same setup, and we make a sudden change in the environment half-way. It takes almost 60 timestamp to converge again, and along the way accumulated large regrets. This is because after the first convergence, our model already has strong belief and it's much harder and slower to update.


### Solutions
We will focus on one of the solutions to this problem. 

The dLinUCB algorithm proposed by Wu et al in 
__[Learning Contextual Bandits in a Non-stationary Environment](https://arxiv.org/pdf/1805.09365.pdf)__ 


## Background
One of the popular algorithms for contextual bandit is the LinUCB.

$E[r_{t,a}|x_{t,a}] = x_{t,a}^{T}\theta_{a}^{*}$

x is context features, $\theta$ is weights. So assumes the expected reward for an action under given context is a linear combination of features.

Weights are calculated by a ridge regression over all the previous contexts encountered vs observed reward


## References
[1] Lihong Li, Wei Chu, John Langford, and Robert E Schapire. 2010. A contextualbandit
approach to personalized news article recommendation. In Proceedings of
19th WWW. ACM, 661–670.

[2]Wu, Q., Iyer, N. and Wang, H., 2018. Learning Contextual Bandits in a Non-stationary Environment. arXiv preprint arXiv:1805.09365.

In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt
from numpy.linalg import inv
plt.style.use('seaborn-whitegrid')

#Generate ground truth models
#Here we assume the reward is linear in the context.
#each action has a theta which is the weights for the context feature vector
np.random.seed(0)


In [6]:
#Base bandit class
class Agent():
    def __init__(self, n_feature, n_action):
        self.thetas = np.zeros((n_action,n_feature))
        self.n_a = n_action
        self.n_f = n_feature
    
    def pick_action(self,x):
        pass
    
    def update_model(self,x,a,r):
        pass

#random bandit
class RandomBandit(Agent):
    def __init__(self, n_feature, n_action):
        Agent.__init__(self,n_feature, n_action)
        
    def pick_action(self,x):
        return np.random.randint(0,self.n_a)

#LinUCB
class LinUCBBandit(Agent):
    def __init__(self, n_feature, n_action):
        Agent.__init__(self,n_feature, n_action)
        self.Ds = [np.zeros(self.n_f).reshape(1,self.n_f)] * self.n_a #exp
        self.Cs = [0] * self.n_a #rewards
        
    def pick_action(self,x):
        delta = 0.3
        max_E = np.NINF
        best_a = 0
        for a in range(self.n_a):
            D = self.Ds[a]
            A = np.dot(D.T,D) + np.identity(self.n_f)
            A_inverse = inv(A)
            alpha = 1. + np.sqrt(np.log(2./delta)/2.)
            bound = alpha * np.sqrt(np.dot(np.dot(x.T, A_inverse), x))
            E = np.dot(x.T,self.thetas[a]) + bound
            if E > max_E:
                max_E = E
                best_a = a

        return best_a
    
    def update_model(self,x,a,r):
        d = len(x)
        self.Ds[a] = np.append(self.Ds[a],[x],axis=0)
        self.Cs[a] = np.append(self.Cs[a],r)
        D = self.Ds[a]
        C = self.Cs[a]
        inverse = inv(np.dot(D.T, D) + np.identity(d))
        new_theta = np.dot(np.dot(inverse,D.T), C)
        self.thetas[a] = new_theta
        
#Thompson Sampling with Bayesian Regression
class ContextualTSAgent(Agent):
    def __init__(self, n_feature, n_action):
        Agent.__init__(self,n_feature, n_action)
        self.exp_dict = {} # experience dict for each action
        
    def pick_action(self,x):
        r_list = []
        for a in range(self.n_a):
            r_list.append(self.get_reward_sample(x,a))
        return np.argmax(r_list)
    
    def get_reward_sample(self,x,a):
        r = 0.
        if a not in self.exp_dict: 
            return r
        #hard coded formula for 3 variables for now.
        with pm.Model() as model:
            pm.GLM.from_formula('r ~ x_0 + x_1 + x_2', data=self.exp_dict[a])
            traces = pm.sample(draws=1, chains=1, tune=100, compute_convergence_checks=False)
            params = traces[0]
            r = x[0] * params['x_0'] + x[1] * params['x_1'] + x[2] * params['x_2'] + params['Intercept']
        return r
    
    def update_model(self,x,a,r):
        new_df = pd.DataFrame({**{'x_{}'.format(i): x[i] for i in range(len(x))}, "r": r},index=[0])
        if a not in self.exp_dict:
            self.exp_dict[a] = new_df
        else:
            self.exp_dict[a] = pd.concat([new_df, self.exp_dict[a]])
    

In [7]:
#class that runs simulation and plot regrets
class Experiment():
    def __init__(self, iter, agent, n_f, n_a, non_stationary):
        self.agent = agent
        self.iterations = iter
        self.non_stationary = non_stationary
        self.n_feature = n_f
        self.n_action = n_a
        self.true_thetas = None
        
    def get_true_reward(self, x,a):
        return np.dot(x,self.true_thetas[a]) + np.random.rand() * 0.5
    
    def get_rand_context(self):
        return np.random.randint(0,5,(self.n_feature))
    
    def gen_true_thetas(self):
        self.true_thetas = np.random.rand(self.n_action,self.n_feature) * 3. - 1.5
        print('Ground truth thetas')
        for a in range(self.n_action):
            print('action ' + str(a) + ': ' + str(self.true_thetas[a]))
    
    def get_optimal_action(self,x):
        return np.argmax([self.get_true_reward(x,a) for a in range(self.n_action)])
    
    def run(self):
        print('\n\nExperiment Starts')
        np.random.seed(0)
        regret = 0.
        regret_list = []
        t_change = np.floor(self.iterations * 0.5)
        self.gen_true_thetas()
        for t in range(self.iterations):
            context = self.get_rand_context()
            a = self.agent.pick_action(context)
            r = self.get_true_reward(context,a)
            self.agent.update_model(context,a,r)
            regret += abs(r - self.get_true_reward(context,self.get_optimal_action(context)))
            regret_list.append(regret)

            if self.non_stationary and t == t_change:
                print('env changed!')
                self.gen_true_thetas()

        print('Estimated thetas')
        for a in range(self.agent.n_a):
            print('action ' + str(a) + ': ' + str(self.agent.thetas[a]))
        
        fig,ax = plt.subplots()
        ax.set_title('Cumulative Regret' if not self.non_stationary else 'Cumulative Regret(Non-stationary Env)')
        ax.set_xlabel('t')
        ax.set_ylabel('regret')
        ax.set_ylim(0.,400.)
        ax.plot(regret_list, label=str(type(self.agent).__name__))
        if self.non_stationary:
            ax.axvline(x=t_change,linestyle=':',color='green')
            ax.text(t_change,0.,'Env Change')
        ax.legend()

        return

In [ ]:
#Main
ITERATIONS = 12
FEATURES = 3
ACTIONS = 5
exp1 = Experiment(ITERATIONS, LinUCBBandit(FEATURES, ACTIONS),FEATURES, ACTIONS, False)
exp2 =  Experiment(ITERATIONS, LinUCBBandit(FEATURES, ACTIONS),FEATURES, ACTIONS, True)
exp1.run()
exp2.run()
agent = ContextualTSAgent(FEATURES, ACTIONS)
exp3 = Experiment(ITERATIONS,agent ,FEATURES, ACTIONS, False)
exp3.run()
            



Experiment Starts
Ground truth thetas
action 0: [0.14644051 0.6455681  0.30829013]
action 1: [ 0.13464955 -0.2290356   0.43768234]
action 2: [-0.18723837  1.175319    1.39098828]
action 3: [-0.34967544  0.87517511  0.08668476]
action 4: [ 0.20413368  1.27678991 -1.28689183]
Estimated thetas
action 0: [0.13250075 0.57163241 0.42525519]
action 1: [ 0.16078104 -0.17650205  0.        ]
action 2: [0.02325049 1.07407788 1.43098307]
action 3: [0.22717899 0.15145266 0.15145266]
action 4: [0.15951095 1.30744332 0.        ]


Experiment Starts
Ground truth thetas
action 0: [0.14644051 0.6455681  0.30829013]
action 1: [ 0.13464955 -0.2290356   0.43768234]
action 2: [-0.18723837  1.175319    1.39098828]
action 3: [-0.34967544  0.87517511  0.08668476]
action 4: [ 0.20413368  1.27678991 -1.28689183]
env changed!
Ground truth thetas
action 0: [-1.14381684 -0.54605046 -0.25721102]
action 1: [-1.30755751  0.57741636  0.19980436]
action 2: [-0.70383153  0.06974416 -1.21817847]
action 3: [ 0.22783949  

Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd, x_2, x_1, x_0, Intercept]
100%|██████████| 101/101 [00:01<00:00, 55.66it/s]
Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd, x_2, x_1, x_0, Intercept]
100%|██████████| 101/101 [00:01<00:00, 62.18it/s]
Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd, x_2, x_1, x_0, Intercept]
100%|██████████| 101/101 [00:01<00:00, 58.54it/s]
Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd, x_2, x_1, x_0, Intercept]
100%|██████████| 101/101 [00:01<00:00, 62.78it/s]
Only 1 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+a